# This notebook was run in google colab

In [ ]:
#Logging in to hugging face as QWen is a gated model
from huggingface_hub import login
login(token="INSERT_TOKEN_HERE")

In [ ]:
#Importing ppeline from transformers and torch
from transformers import pipeline
import torch

In [ ]:
#Test message for starting the model
messages = [
    {"role": "user", "content": "Who are you?"},
]
#Setting up pipeline with QWen 2.5 1.5B instruct
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-1.5B-Instruct")
pipe(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I am Qwen, an AI language model developed by Alibaba Cloud. I can assist with various tasks'}]}]

# Importing Data

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your file
file_loc = '/content/drive/MyDrive/transcripts_v3'

Mounted at /content/drive


In [ ]:
files = [file for file in os.listdir(file_loc) if ".txt" in file]

transcripts = []

for file in files:
    with open(os.path.join(file_loc,file), 'r') as file:
        # Read the contents of the file
        transcripts.append(file.read())
        #print(contents)

In [ ]:
def extract_member_transcript(transcript):
    transcript = transcript.split("\n\n")

    member_script = []

    for line in transcript:
        line = line.strip()
        if line[:6] == "Member":
            member_script.append(line[8:])

    print(len(transcript),len(member_script))
    return "\n".join(member_script)

member_transcripts = [extract_member_transcript(x) for x in transcripts]

23 11
14 7
20 10
22 11
20 10
12 6
16 8
24 11
20 10
15 6
16 8
16 8
21 9
18 9
18 9
22 11
21 11
17 8
18 9
20 9
14 7
12 6
18 9
14 7
16 8
16 8
19 10
16 8
22 11
14 7
14 7
23 10
24 12
1 1
24 11
13 7
22 11
16 8
1 1
1 1
12 6
16 8
1 1
21 11
20 10
19 8
12 6
18 9
20 10
30 13
16 8
18 9
1 1
1 1
26 11
18 9
16 8
20 10
21 11
24 11
14 7
20 10
24 12
19 9
18 9
22 10
33 15
22 10
15 8
18 9
16 8
16 8
18 9
20 10
16 8
23 10
1 1
1 1
12 6
1 1
12 5
15 8
14 7
19 9
34 15
22 11
17 9
15 8
14 1
14 7
15 8
14 7
24 12
15 8
21 10
20 10
18 9
1 1
17 7
17 8
17 8
16 8
14 6
24 12
17 8
14 7
18 9
14 7
14 7
15 7
14 7
21 11
14 7
18 9
18 9
27 13
16 8
21 9
21 11
14 7
14 7
12 6
16 7
20 10
14 7
17 9
21 10
14 7
14 7
17 8
12 1
21 6
20 10
19 10
16 8
14 6
26 11
17 8
20 10
19 10
16 8
16 8
14 7
18 9
18 9
15 7
16 7
1 1
16 7
18 9
1 1
38 16
19 10
1 1
14 7
13 6
16 8
18 9
18 9
18 9
16 8
16 8
19 10
18 9
16 7
21 10
18 9
18 9
19 10
17 8
19 9
22 11
14 7
19 10
17 9
23 11
14 7
17 7
15 7
18 9
16 8
19 10
15 7
19 10
20 10
15 8
23 11
19 8
16 8
21 10
24 12

# Clasify Resolved or Follow Up Needed

In [ ]:

messages = []


for mts in member_transcripts:
  mess = f"""I have attached the member portion of a customer support call log.:
      Start of log:
      {mts}
      End of Log
       please identify the outcome of the call, if it was resolved, or if follow up action is needed.
       the output should be 1 for resolved and 0 for action needed. Start your response with the reason
       why you chose that outcome, and then end with 0 or 1"""
  messages.append({"role": "user", "content": mess})

In [ ]:
results = []

#Propmting qwen model pipeline and saving reuslts
for mess in messages:
  res=pipe([mess], max_length=2500)
  P2 = res[0]["generated_text"]
  P2.append(
{'role': 'user',
'content': 'given your explanation of your response, please reply with the response you have chosen only, so respond with 1 for resolved or 0 otherwise'})
  results.append(pipe(P2, max_length = 2500))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
#Extracting final results
resolved = [x[0]["generated_text"][3]["content"] for x in results]

# Sentiment

In [ ]:
#Importing necessary packages from transformers
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

from scipy.special import softmax

#Loading cardiff NLP roberta model
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Function to run sentiment model
def run_trs_model(text):
    #Specifying truncation to only-first as later parts of the call are likely more indicative
    encoded_input = tokenizer(text, return_tensors='pt', truncation="only_first", max_length=511)
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    return scores

In [ ]:
trs_sentiments = [run_trs_model(ts) for ts in member_transcripts]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

# Combining

In [ ]:
#Extracting relevant sentiment scores
negative = [x[0] for x in trs_sentiments]
neutral = [x[1] for x in trs_sentiments]
positive = [x[2] for x in trs_sentiments]

In [ ]:
#Saving resolved and sentiment predictions as a dataframe
df = pd.DataFrame({
    'file':files,
    'resolved': resolved,
    'negative': negative,
    'neutral': neutral,
    'positive': positive
})


In [ ]:
#Writing the dataframe to CSV
df.to_csv(os.path.join("/content/drive/MyDrive/", "LLM_Outputs_v2.csv"))